In [ ]:
pip install google-auth
pip install google-cloud-bigquery[pandas]

In [3]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
key_path = "../../Downloads/bruin-case-credentials.json"


In [4]:
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [25]:
# 1- What are the most popular repos?

q1 = """
CREATE OR REPLACE TABLE `bruin-hiring.cdt_caneraslan.q1` AS
(
  SELECT
    repo_name,
    COUNT(repo_name) AS commit_number
  FROM `bruin-hiring.github_repos.sample_commits`
  GROUP BY repo_name
  ORDER BY commit_number DESC
)
"""

res = client.query(q1).result()
# df = res.to_dataframe()
# print(df)

In [14]:
# 2- Who are the top contributors?

q2 = """
CREATE OR REPLACE TABLE `bruin-hiring.cdt_caneraslan.q2` AS
(
SELECT
  committer.name as contributor,
  COUNT(committer.email) as contribution_number
FROM `bruin-hiring.github_repos.sample_commits`
GROUP BY contributor, committer.email
ORDER BY contribution_number DESC
)
"""

query_job = client.query(q2)
res = query_job.result()

# df = res.to_dataframe()
# print(df)

In [26]:
# 3- Are there any developer contributing to more than 1 repos?

q3 = """
CREATE OR REPLACE TABLE `bruin-hiring.cdt_caneraslan.q3` AS
(
SELECT 
  committer.name as contributor, 
  COUNT(DISTINCT repo_name) as distinct_repo_number  
FROM `bruin-hiring.github_repos.sample_commits`  
GROUP BY 1
having count(distinct repo_name)>1
ORDER BY distinct_repo_number DESC
)
"""

res = client.query(q3).result()
# print(res.to_dataframe())


In [19]:
# 4- What are the most popular email domains?

q4 = """
CREATE OR REPLACE TABLE `bruin-hiring.cdt_caneraslan.q4` AS
(
SELECT SPLIT(committer.email, '@')[SAFE_OFFSET(1)] AS email_domain, COUNT(committer.email) as commit_number
FROM `bruin-hiring.github_repos.sample_commits`
WHERE committer.email is not null
GROUP BY email_domain
ORDER BY commit_number DESC
)
"""

res = client.query(q4).result()
# print(res.to_dataframe())

In [22]:
# 5- Number of commits per day

q5 = """
CREATE OR REPLACE TABLE `bruin-hiring.cdt_caneraslan.q5` AS
(
SELECT 
  DATE(committer.date) AS commit_date, 
  COUNT(committer.date) as commit_number 
FROM `bruin-hiring.github_repos.sample_commits`
GROUP BY commit_date
ORDER BY commit_number DESC
)
"""

res = client.query(q5).result()
# print(res.to_dataframe())

In [24]:
# 6- Number of commits per weekday

q6 = """
CREATE OR REPLACE TABLE `bruin-hiring.cdt_caneraslan.q6` AS
(
SELECT 
    EXTRACT(DAYOFWEEK FROM committer.date) AS commit_date,
    COUNT(committer.date) AS commit_number
FROM bruin-hiring.github_repos.sample_commits
GROUP BY 1
ORDER BY 1 DESC
)
"""

res = client.query(q6).result()
# print(res.to_dataframe())

In [18]:
q8 = """
CREATE OR REPLACE TABLE bruin-hiring.cdt_caneraslan.q8 AS
(
WITH ranked_paths AS (
  SELECT
    repo_name,
    difference.old_path,
    COUNT(difference.old_path) AS path_count,
    ROW_NUMBER() OVER (PARTITION BY repo_name ORDER BY COUNT(difference.old_path) DESC) AS rank
  FROM
    `bruin-hiring.github_repos.sample_commits`,
    UNNEST(difference) AS difference
  GROUP BY
    repo_name, difference.old_path
)

SELECT
  repo_name,
  old_path,
  path_count
FROM
  ranked_paths
WHERE
  rank <= 5
ORDER BY
  repo_name, path_count DESC
)
"""

res = client.query(q8).result()
# print(res.to_dataframe())
